# In-game data predictions experiments (supportive regression on init dataset)

In [1]:
import pandas
import psycopg2

player_ranks_to_mmr_dict = {
  "Herald IW": [1, 1],
  "Herald I": [1, 154],
  "Herald II": [154, 308],
  "Herald III": [308, 462],
  "Herald IV": [462, 616],
  "Herald V": [616, 769],
  "Guardian IW": [770, 770],
  "Guardian I": [770, 924],
  "Guardian II": [924, 1078],
  "Guardian III": [1078, 1232],
  "Guardian IV": [1232, 1386],
  "Guardian V": [1386, 1540],
  "Crusader IW": [1540, 1540],
  "Crusader I": [1540, 1694],
  "Crusader II": [1694, 1848],
  "Crusader III": [1848, 2002],
  "Crusader IV": [2002, 2156],
  "Crusader V": [2156, 2310],
  "Archon IW": [2310, 2310],
  "Archon I": [2310, 2464],
  "Archon II": [2464, 2618],
  "Archon III": [2618, 2772],
  "Archon IV": [2772, 2926],
  "Archon V": [2926, 3080],
  "Legend IW": [3080, 3080],
  "Legend I": [3080, 3234],
  "Legend II": [3234, 3388],
  "Legend III": [3388, 3542],
  "Legend IV": [3542, 3696],
  "Legend V": [3696, 3850],
  "Ancient IW": [3850, 3850],
  "Ancient I": [3850, 4004],
  "Ancient II": [4004, 4158],
  "Ancient III": [4158, 4312],
  "Ancient IV": [4312, 4466],
  "Ancient V": [4466, 4620],
  "Divine IW": [4620, 4620],
  "Divine I": [4620, 4820],
  "Divine II": [4820, 5020],
  "Divine III": [5020, 5220],
  "Divine IV": [5220, 5420],
  "Divine V": [5420, 5620],
  "Immortal IW": [5621, 5621],
  "Immortal I": [5622, 5820],
  "Immortal II": [5820, 6020],
  "Immortal III": [6020, 6320],
  "Immortal IV": [6320, 6620]
}

def transform_to_mmr(player_rank):
    return sum(player_ranks_to_mmr_dict[player_rank]) / 2

query = """
    SELECT
    ms.*,
    h.name_local as hero_name_local,
    h.hero_pickrate_average,
    h.hero_winrate_average,
    h.hero_pickrate_up_to_crusader,
    h.hero_winrate_up_to_crusader,
    h.hero_pickrate_archon,
    h.hero_winrate_archon,
    h.hero_pickrate_legend,
    h.hero_winrate_legend,
    h.hero_pickrate_ancient,
    h.hero_winrate_ancient,
    h.hero_pickrate_divine_immortal,
    h.hero_winrate_divine_immortal,
    m.datetime as match_datetime,
    m.radiant_win as match_radiant_win,
    m.duration as match_duration,
    m.radiant_score as match_radiant_score,
    m.dire_score as match_dire_score
FROM
    match_stats ms
JOIN
    heroes h ON ms.hero_id = h.id
JOIN
    matches m ON ms.match_id = m.id;

"""

conn = psycopg2.connect(
                    dbname='dota_ai_od',
                    user='limited_user',
                    password='*removed*',
                    host='*azure*',
                    port='5432')


df = pandas.read_sql_query(query, conn)

df['player_side'] = df['player_side'].astype('category')


conn.close()

C:\Users\nicky\AppData\Local\Temp\ipykernel_19112\4066457769.py:95: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pandas.read_sql_query(query, conn)


In [2]:
df['match_id'] = df['match_id'].astype('int64')
df['player_nickname'] = df['player_nickname'].astype(str)
df['player_side'] = df['player_side'].astype('category')
df['player_lasthits'] = df['player_lasthits'].astype('int64')
df['player_denies'] = df['player_denies'].astype('int64')
df['hero_name_local'] = df['hero_name_local'].astype(str)
df['player_match_rank_initial'] = df['player_match_rank_initial'].astype('category')
df['player_id'] = df['player_id'].astype('int64')

In [3]:
df['player_match_rank_initial_mmr'] = df['player_match_rank_initial'].apply(transform_to_mmr)
df['player_match_rank_initial_mmr'] = df['player_match_rank_initial_mmr'].astype('float64')


winrate_columns = [col for col in df.columns if 'winrate' in col.lower() and 'hero' not in col.lower()]
pickrate_columns = [col for col in df.columns if 'pickrate' in col.lower() and 'hero' not in col.lower()]

df[winrate_columns] *= 100
df[pickrate_columns] *= 100

df['player_hero_winrate_overall'] *= 100

df.head(100)

,match_id,player_id,hero_id,player_q_mmr_diff,player_hero_winrate_overall,player_hero_total_matches_played,dire_winrate_all_time,dire_games_played_all_time,radiant_winrate_all_time,radiant_games_played_all_time,...,hero_pickrate_ancient,hero_winrate_ancient,hero_pickrate_divine_immortal,hero_winrate_divine_immortal,match_datetime,match_radiant_win,match_duration,match_radiant_score,match_dire_score,player_match_rank_initial_mmr
0,7398770906,363104336,54,-154,50.000000,4,52.890176,346,50.381680,393,...,19.0338,54.9159,19.3351,53.8362,2023-10-25 14:14:53,False,3175,55,54,1925.0
1,7398770906,204634639,100,154,42.553192,47,48.453608,291,52.365930,317,...,6.1267,47.2815,7.5739,48.2371,2023-10-25 14:14:53,False,3175,55,54,2233.0
2,7398770906,144595206,36,-462,52.252250,333,45.172307,3221,51.850670,3134,...,8.2786,49.4327,5.0972,47.9513,2023-10-25 14:14:53,False,3175,55,54,1617.0
3,7398770906,167763814,84,-308,57.425743,101,47.641712,2311,51.565146,2364,...,9.6141,50.8513,6.1195,49.6469,2023-10-25 14:14:53,False,3175,55,54,1771.0
4,7398770906,1514889200,85,0,58.695650,46,44.047618,504,56.448203,473,...,6.5329,52.2018,6.5435,50.8980,2023-10-25 14:14:53,False,3175,55,54,2079.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7418150617,385421629,76,-1371,30.434780,23,43.255815,645,48.826292,639,...,6.7161,50.0457,6.1592,49.4761,2023-11-04 17:03:13,True,1616,39,17,692.5
96,7418150617,879201523,138,15,59.375000,64,47.456214,1199,54.567310,1248,...,3.5713,45.5668,3.6903,47.0845,2023-11-04 17:03:13,True,1616,39,17,2079.0
97,7418150617,136311244,137,1401,62.068963,29,46.631017,1870,53.201133,1765,...,5.2408,47.5735,7.0162,49.2753,2023-11-04 17:03:13,True,1616,39,17,3465.0
98,7418150617,136561541,99,477,28.571430,21,47.729987,1674,52.736900,1699,...,5.7045,50.0346,5.3149,50.0514,2023-11-04 17:03:13,True,1616,39,17,2541.0


In [4]:
def get_hero_pickrate_winrate_for_rank(group):
    closest_player_idx = group['player_q_mmr_diff'].abs().idxmin()

    closest_player_rank = group.loc[closest_player_idx, 'player_match_rank_initial'].split(' ')[0]
    
    if closest_player_rank.lower() in ['herald', 'guardian', 'crusader']:
      closest_player_rank = 'up_to_crusader'

    pickrate_column = f'hero_pickrate_{closest_player_rank.lower()}'
    winrate_column = f'hero_winrate_{closest_player_rank.lower()}'
    group['hero_pickrate_for_rank'] = group[pickrate_column]
    group['hero_winrate_for_rank'] = group[winrate_column]
    
    return group

df_with_pickrate = df.groupby('match_id').apply(get_hero_pickrate_winrate_for_rank)

df_with_pickrate['player_heroes_pick_confidence_score_total'] = df_with_pickrate['player_heroes_pick_confidence_score_allies'] + df_with_pickrate['player_heroes_pick_confidence_score_enemies']
df_with_pickrate['player_matches_abandonment_rate'] = (df_with_pickrate['player_matches_abandoned'] / df_with_pickrate['player_all_matches_played_number']) * 100


df_with_pickrate.reset_index(drop=True, inplace=True)

df_with_pickrate['has_team_won'] = df_with_pickrate.groupby(['match_id', 'player_side'])['player_has_won'].transform('first')

C:\Users\nicky\AppData\Local\Temp\ipykernel_19112\1974297312.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_with_pickrate = df.groupby('match_id').apply(get_hero_pickrate_winrate_for_rank)
C:\Users\nicky\AppData\Local\Temp\ipykernel_19112\1974297312.py:24: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_with_pickrate['has_team_won'] = df_with_pickrate.groupby(['match_id', 'player_side'])['player_has_won'].transform('first')


In [5]:
df_with_pickrate.head(100)

,match_id,player_id,hero_id,player_q_mmr_diff,player_hero_winrate_overall,player_hero_total_matches_played,dire_winrate_all_time,dire_games_played_all_time,radiant_winrate_all_time,radiant_games_played_all_time,...,match_radiant_win,match_duration,match_radiant_score,match_dire_score,player_match_rank_initial_mmr,hero_pickrate_for_rank,hero_winrate_for_rank,player_heroes_pick_confidence_score_total,player_matches_abandonment_rate,has_team_won
0,7393139706,168352898,103,-631,0.000000,5,47.937410,1406,52.932550,1364,...,False,3368,41,48,2233.0,1.3819,50.2281,0,1.752190,False
1,7393139706,299921151,74,139,44.000000,25,57.120000,1250,57.441860,1290,...,False,3368,41,48,3003.0,17.7897,46.7979,0,1.633554,False
2,7393139706,356518944,7,-785,65.517240,29,57.670456,352,55.466664,375,...,False,3368,41,48,2079.0,11.1720,49.9146,0,0.434153,False
3,7393139706,132549973,104,139,66.666670,39,49.499545,1099,54.900180,1102,...,False,3368,41,48,3003.0,14.1323,49.7264,0,5.283505,False
4,7393139706,1034923441,70,601,59.375000,32,49.844238,1284,52.788390,1309,...,False,3368,41,48,3465.0,7.4670,49.4052,0,0.319872,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7393596207,318590696,21,-770,68.421054,57,54.566210,438,57.049894,461,...,True,2894,54,50,2079.0,11.8661,46.7467,0,0.455927,False
96,7393596207,1225720384,54,462,44.444445,9,48.876405,712,56.133336,750,...,True,2894,54,50,3311.0,16.2930,53.9691,0,0.392157,False
97,7393596207,407552781,2,154,63.157890,19,47.140960,1504,55.423373,1429,...,True,2894,54,50,3003.0,18.7621,53.5622,1,0.937346,False
98,7393596207,156932492,83,0,65.372170,309,48.115440,4643,53.328866,4476,...,True,2894,54,50,2849.0,4.4050,52.3476,2,3.575990,False


In [6]:
df = df_with_pickrate.drop(columns=['match_datetime', 'hero_id', 'player_nickname', 'player_match_rank_initial', 'average_match_mmr', 'match_id', 'player_id', 'player_has_won', 'match_radiant_win', 'match_duration', 'match_radiant_score', 'match_dire_score', 'player_side', 'hero_name_local', 'has_team_won', 'hero_pickrate_up_to_crusader', 'hero_winrate_up_to_crusader', 'hero_pickrate_archon', 'hero_winrate_archon', 'hero_pickrate_legend', 'hero_winrate_legend', 
                                    'hero_pickrate_ancient', 'hero_winrate_ancient', 'hero_pickrate_divine_immortal', 'hero_winrate_divine_immortal'])

df.describe()

,player_q_mmr_diff,player_hero_winrate_overall,player_hero_total_matches_played,dire_winrate_all_time,dire_games_played_all_time,radiant_winrate_all_time,radiant_games_played_all_time,player_heroes_pick_confidence_score_allies,player_heroes_pick_confidence_score_enemies,player_time_played_all_matches,...,player_hero_damage_average_all_matches,player_tower_damage_average_all_matches,player_hero_healing_average_all_matches,hero_pickrate_average,hero_winrate_average,player_match_rank_initial_mmr,hero_pickrate_for_rank,hero_winrate_for_rank,player_heroes_pick_confidence_score_total,player_matches_abandonment_rate
count,42100.000000,42100.000000,42100.000000,42100.000000,42100.000000,42100.000000,42100.000000,42100.000000,42100.000000,4.210000e+04,...,42100.000000,42100.000000,42100.000000,42100.000000,42100.000000,42100.000000,42100.000000,42100.000000,42100.000000,42100.000000
mean,-0.002993,51.282409,112.025677,47.608343,1626.302138,53.308479,1625.710974,0.060285,0.068931,8.061033e+06,...,23356.802936,2682.087038,869.251196,10.528974,49.841891,2655.786390,11.008307,49.937926,0.129216,1.432761
std,664.032867,11.766337,215.277311,2.632630,1180.214991,2.664362,1179.080342,0.814180,1.359543,5.854762e+06,...,6465.874590,1071.183970,518.474158,5.291055,2.197483,737.244701,5.921863,2.626255,1.718393,1.689259
min,-2635.000000,0.000000,1.000000,16.666667,6.000000,28.571430,7.000000,-4.000000,-6.000000,3.180700e+04,...,8510.794000,289.058840,0.000000,1.271280,43.877400,77.500000,0.810500,41.840900,-8.000000,0.000000
25%,-416.000000,46.000000,22.000000,46.086619,738.000000,51.829972,736.000000,0.000000,-1.000000,3.632462e+06,...,20210.308250,1911.763125,567.622202,6.681380,47.889340,2233.000000,6.836300,47.717600,-1.000000,0.535380
50%,-31.000000,51.685390,51.000000,47.330353,1369.000000,53.106576,1369.000000,0.000000,0.000000,6.784253e+06,...,23146.551000,2545.683350,758.883455,9.508720,49.921240,2695.000000,9.915000,50.038400,0.000000,1.008742
75%,416.000000,57.142860,116.000000,48.818317,2236.000000,54.545456,2241.000000,0.000000,1.000000,1.110377e+07,...,26223.828250,3285.868750,1024.871325,13.175980,51.453660,3157.000000,14.184700,52.124600,1.000000,1.769088
max,3172.000000,100.000000,7250.000000,81.818180,15156.000000,81.506850,15377.000000,5.000000,9.000000,7.661488e+07,...,932722.440000,9858.976000,10920.932000,28.757380,54.907300,5621.000000,29.932700,55.537800,14.000000,71.830986


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

columns_to_drop = ['player_xpm', 'player_denies', 'player_lasthits',
                   'player_assists_number', 'player_kills_number', 'player_net', 'hero_lvl', 'player_deaths_number']

temp_df = df.drop(columns=columns_to_drop).head(10000)

temp_df.head(10)

,player_q_mmr_diff,player_hero_winrate_overall,player_hero_total_matches_played,dire_winrate_all_time,dire_games_played_all_time,radiant_winrate_all_time,radiant_games_played_all_time,player_heroes_pick_confidence_score_allies,player_heroes_pick_confidence_score_enemies,player_time_played_all_matches,...,player_hero_damage_average_all_matches,player_tower_damage_average_all_matches,player_hero_healing_average_all_matches,hero_pickrate_average,hero_winrate_average,player_match_rank_initial_mmr,hero_pickrate_for_rank,hero_winrate_for_rank,player_heroes_pick_confidence_score_total,player_matches_abandonment_rate
0,-631,0.000000,5,47.937410,1406,52.932550,1364,0,0,6606301,...,27282.379,3840.0867,189.48804,1.65384,50.79214,2233.0,1.3819,50.2281,0,1.752190
1,139,44.000000,25,57.120000,1250,57.441860,1290,0,0,6144606,...,27830.834,3779.7305,665.06130,16.13098,47.44248,3003.0,17.7897,46.7979,0,1.633554
2,-785,65.517240,29,57.670456,352,55.466664,375,0,0,1838111,...,15426.104,890.8892,1380.35660,11.20572,50.51110,2079.0,11.1720,49.9146,0,0.434153
3,139,66.666670,39,49.499545,1099,54.900180,1102,0,0,5247812,...,26371.805,4444.6070,1363.43500,12.08786,49.25336,3003.0,14.1323,49.7264,0,5.283505
4,601,59.375000,32,49.844238,1284,52.788390,1309,0,0,6457179,...,22130.459,2527.6013,812.32550,8.01404,49.80836,3465.0,7.4670,49.4052,0,0.319872
5,-477,62.264150,106,46.742857,875,52.662724,845,1,0,4258488,...,25070.113,2954.2983,1113.50820,16.88724,49.62226,2387.0,16.4077,49.5248,1,1.498929
6,-323,27.272728,11,48.480000,1250,51.566470,1181,1,0,5937946,...,28541.140,2986.3167,588.93910,8.46242,49.90816,2541.0,8.4002,50.3744,1,1.272149
7,755,41.176470,34,50.423920,2241,51.577955,2123,1,0,10538873,...,25304.502,3413.4014,1398.35380,9.50872,52.06560,3619.0,8.5947,52.3047,1,0.800971
8,601,50.684930,73,47.902870,3171,52.808990,3204,1,0,15674693,...,25427.800,2425.9143,961.92395,9.74936,47.35170,3465.0,8.8040,46.9220,1,1.350640
9,-15,55.670100,97,51.094890,1918,51.021504,1860,0,1,9263806,...,26473.541,3892.5752,754.53485,17.28832,53.77350,2849.0,16.2930,53.9691,1,1.644737


In [8]:
X = temp_df.drop(columns=['player_gpm'])
y = temp_df['player_gpm']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=80, random_state=42)

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Mean Squared Error:', mse)
print('R-squared:', r2)

Mean Squared Error: 14800.631810781251
R-squared: 0.30094965159054066


## Conclusion

LinearRegression-based experiments conducted in scope of this notebook (one of which is presented) were unsuccessful.
Usage of an advanced model may have given some better results on this non-linear data but that approach would go beyond the scope of my research.

Additionally: I doubt those results (even with advanced models) would do any better than around 50% accuracy.  